In [1]:
import pandas as pd
import hces

df = hces.load()

---

### Elegibility - households with LPG; treatment - got LPG subsidy

In [2]:
xdf = df[df['energy_source_cooking'] == 'LPG'].copy()
xdf['treat'] = xdf['received_subsidy_lpg'] == 1

In [3]:
hces.compare(xdf, ['sector', 'treat'], ['FOOD', 'EDUCATION', 'HEALTH'])

FOOD    EDUCATION      HEALTH
sector treat                                       
rural  False  10783.316782  1000.952985  433.425004
       True   10304.244202  1077.460301  388.674376
urban  False  12302.774650  1214.521744  434.652115
       True   11456.505997  1308.326017  402.744170

In [4]:
hces.compare(xdf, ['sector', 'treat'], hces.EDU_COLS)

books_1st  books_2nd  stationery        fees    coaching  \
sector treat                                                              
rural  False  181.454195   6.454990  107.155802  556.564321  117.206238   
       True   195.943441   6.303216  120.719899  546.368484  184.091256   
urban  False  233.017976   8.945897  116.784912  618.747775  190.698963   
       True   251.554819   7.474849  126.165810  607.593996  277.255615   

              edu_other  
sector treat             
rural  False  32.117439  
       True   24.034005  
urban  False  46.326221  
       True   38.280928

In [5]:
hces.compare(xdf, ['sector', 'treat'], hces.FOOD_COLS)

cereals      pulses  sugar_salt        dairy          veg  \
sector treat                                                                  
rural  False  1126.300339  464.270172  202.767808  2146.789258  1273.407120   
       True   1132.638433  459.172234  181.156115  1920.768315  1282.015771   
urban  False  1225.452929  454.647028  195.051739  2586.013514  1291.629828   
       True   1257.822365  444.871610  167.132154  2118.634500  1317.788106   

                  fruits         meat         oil       spice         bev  \
sector treat                                                                
rural  False  703.852694  1114.013370  608.877947  722.715537  619.949302   
       True   654.377691  1121.928621  586.176392  671.945959  579.407206   
urban  False  861.914723  1052.081047  587.536204  733.406543  859.044144   
       True   786.109385  1306.384896  573.068969  680.462212  709.153508   

                   cooked    processed  
sector treat                            
rural  False   995.402019   804.971216  
       True    960.481669   754.175795  
urban  False  1355.982866  1100.014083  
       True   1175.963021   919.115272

In [6]:
hces.compare(xdf, ['sector', 'treat'], hces.HEALTH_COLS)

410        411        412        413        414  \
sector treat                                                           
rural  False  117.677013  61.300292  62.800372  76.706319  27.605683   
       True   103.219823  51.106231  58.067028  73.482343  20.881345   
urban  False  106.330091  61.136936  66.344246  80.709912  25.319098   
       True    94.095501  56.769302  55.715626  80.119535  22.501003   

                    420       421        422       423       424  
sector treat                                                      
rural  False  65.220341  7.138125  12.094292  0.431874  2.450693  
       True   60.979399  8.134439  10.296485  0.429572  2.077709  
urban  False  68.689864  7.986618  14.730733  0.532906  2.871711  
       True   70.648463  7.909721  12.710707  0.633854  1.640457

---

### Treatment = free electricity; eligibility, households with electricity

In [7]:
xdf = df[df['energy_source_lighting'].str.startswith('electricity').fillna(False)].copy()
xdf['treat'] = xdf['received_free_electricity'] == 1

In [8]:
hces.compare(xdf, ['sector', 'treat'], ['FOOD', 'EDUCATION', 'HEALTH'])

FOOD    EDUCATION      HEALTH
sector treat                                       
rural  False   9668.830559   847.809294  347.000672
       True   10441.379775   866.006397  370.846148
urban  False  11647.460165  1231.538564  410.731982
       True   12166.370237  1158.257402  440.722818

In [9]:
hces.compare(xdf, ['sector', 'treat'], hces.EDU_COLS)

books_1st  books_2nd  stationery        fees    coaching  \
sector treat                                                              
rural  False  150.983145   5.156507   94.354093  387.353915  191.023861   
       True   148.190821   6.250337  118.417354  494.751959   66.305731   
urban  False  236.466095   7.353500  121.143798  577.807462  243.000408   
       True   244.744621  10.725220  124.894999  598.243544  132.709299   

              edu_other  
sector treat             
rural  False  18.937772  
       True   32.090195  
urban  False  45.767301  
       True   46.939720

In [10]:
hces.compare(xdf, ['sector', 'treat'], hces.FOOD_COLS)

cereals      pulses  sugar_salt        dairy          veg  \
sector treat                                                                  
rural  False  1063.722725  428.408157  184.256231  1763.322041  1277.150314   
       True   1160.043186  417.918720  201.612692  2154.728579  1163.217224   
urban  False  1160.750198  423.727923  178.400241  2309.127075  1244.823945   
       True   1224.754043  445.650890  175.351800  2412.241672  1232.639493   

                  fruits         meat         oil       spice         bev  \
sector treat                                                                
rural  False  571.151379  1068.980119  605.280203  665.307665  475.841094   
       True   625.802380   904.814725  536.818336  682.632355  612.427229   
urban  False  798.825029  1042.888950  574.467099  676.488997  779.503672   
       True   869.310940  1144.353905  515.162987  698.789893  878.118563   

                   cooked    processed  
sector treat                            
rural  False   872.355116   693.055514  
       True   1256.094884   725.269466  
urban  False  1434.989964  1023.467070  
       True   1581.174468   988.821583

In [11]:
hces.compare(xdf, ['sector', 'treat'], hces.HEALTH_COLS)

410        411        412        413        414  \
sector treat                                                           
rural  False   94.873296  46.993553  45.352016  62.362947  20.484061   
       True    96.580406  53.393059  53.958120  65.384734  23.521128   
urban  False   95.800224  58.118197  60.955615  79.440296  22.728177   
       True   114.902247  60.809009  64.169723  83.498673  26.836601   

                    420       421        422       423       424  
sector treat                                                      
rural  False  58.747835  6.512813   9.172506  0.411877  2.089771  
       True   57.919529  7.375221  10.828622  0.339232  1.546097  
urban  False  69.217542  7.884871  13.534963  0.566695  2.485403  
       True   65.517427  7.744820  14.757620  0.466093  2.020606